Pull in the data for the water flow 

In [1]:
import pandas as pd
import csv
import requests
import os
from bs4 import BeautifulSoup
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# change to the path with the raw csv file
path=r'C:\Springboard\Github\River_Analysis'
os.chdir(path)

In [3]:
# abstrated functions first before starting the loop. No need to redefine them every time
# function for pulling the date range for reach USGS gage
# takes in string of the gage number with USGS
def pull_start(USGS):
    link = "https://waterdata.usgs.gov/nwis/dv?referred_module=sw&site_no=" + USGS
    response = requests.get(link)
    soup = BeautifulSoup(response.text, "html.parser")
    site_sum = soup.find_all(id='available_parameters_table')
    rows_sum = site_sum[0].find_all('tr')
    for row in rows_sum:
        cell_sum = row.find_all('td')
        for i, cell in enumerate(cell_sum):
            if 'discharge' in cell.get_text().lower():
                # get start date from the next cell
                start_date = re.findall(r'\d{4}-\d{2}-\d{2}', cell_sum[i+1].get_text())[0]
                end_date = re.findall(r'\d{4}-\d{2}-\d{2}', cell_sum[i+2].get_text())[0]
                # re.findall(r'\d{4}-\d{2}-\d{2}', cell[next].get_text())[0]
                return start_date, end_date
    return 0, 0

In [4]:
# this function pulls the water date from a given gage from start_date to end_date
def pull_water(USGS, start_date, end_date):
    link_begin = 'https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format=rdb&site_no='
    link_mid = '&referred_module=sw&period=&begin_date='
    link_end = '&end_date='
    link = link_begin + USGS + link_mid + start_date + link_end + end_date
    response = requests.get(link)
    # raw_file = USGS + '.csv'
    with open(raw_file, 'wb') as f:
        for chunk in response:
            f.write(chunk)

# this gets a raw file of the data

In [5]:
def cleaning_file(gage, raw):
    # read the raw CSV back in and remove the commented lines
    # open raw CSV
    fi = open(raw, 'r')

    # read raw CSV to clean CSV - eliminate comment rows with "#"
    clean_file = gage + '_clean.csv'
    with open(clean_file, 'w') as fo:
        lines = fi.readlines()
        for line in lines:
            if "#" not in line:
                fo.write(line)
    fi.close()
    return clean_file

In [6]:
# pull in the list of USGS gages that I use on rivermaps.co
USGS_list = pd.read_csv('USGS_list.csv', error_bad_lines=False, delimiter='=', names=['gages', 'flows'])

,gages,flows
0,09063000,32
1,09064600,80
2,09067020,146
3,09070000,221
4,09065100,-999999
...,...,...
240,CATCANYON,4320
241,RFBASALT,123
242,ESCCREEKCO,249
243,SFPAYETTEID,1021


In [7]:
# create list of USGS Gages that I care about
gage_list = USGS_list['gages'].tolist()

###  Loop through USGS gages in the list

In [9]:
for GAGE in gage_list:
    # jump to a folder to hold the CSV
    os.chdir(r'C:\Springboard\Github\River_Analysis\CSV_files')
    #GAGE = '14372300'
    raw_file = GAGE + '.csv'
    try:
        # pulls the data for just one gage
        start_date, end_date = pull_start(GAGE)
        pull_water(GAGE, start_date, end_date)
        clean_file = cleaning_file(GAGE, raw_file)
        flow = pd.read_csv(clean_file, error_bad_lines=False, delimiter='\t')
        # drop the first row - that looks like useless data in most gages
        flow.drop(0, inplace=True)
        # rename columns to make for easier dropping
        flow.columns = ['agency', 'gage', 'date', 'y', 'useless']
        # drop everything except the flow ('y') column and the date
        flow.drop(['agency', 'gage', 'useless'], axis=1, inplace=True)
        # make sure the flow is all a floating point number
        flow['y'] = flow['y'].astype(float)
        # let's automatically fill the NaN values with the more recent value (backfill)
        flow['y'].fillna(method='backfill', inplace=True)
        # rename date column for FB prophet (in case we decide to go back to use that analysis package)
        flow.rename(columns={'date':'ds'}, inplace=True)
        # create Month-Day column that is named 'MD'
        # add day and month columns to the existing DF
        flow["Month"] = flow['ds'].str[5:7]
        flow["Day"] = flow['ds'].str[8:10]
        flow['MD'] = flow["Month"] + '-' + flow["Day"]
        # make sure month and day are the correct data types - in case we want to do future analysis on them
        flow['Month'] = flow['Month'].astype(int)
        flow['Day'] = flow['Day'].astype(int)
        # do that group by analysis
        summary = flow.groupby(["MD"], as_index=False)['y'].agg(['mean', 'min', 'median']) # 'max'
        ##  plot the figure and save to file
        # set the figure size
        sns.set(rc={'figure.figsize':(11.7,8.27)})
        # create the plot
        os.chdir(r'C:\Springboard\Github\River_Analysis\Plots')
        g  = sns.lineplot(data=summary) # , x='MD', y=['min', 'mean', 'max', 'median'])
        _ = plt.xticks(['01-01', '02-01', '03-01', '04-01', '05-01', '06-01', '07-01', '08-01', '09-01', '10-01', '11-01', '12-01', '12-31'])
        plt.xlabel('Month-Day')
        plt.ylabel('Daily Average Flow (cfs)')
        plt.title(GAGE)
        plt.savefig(GAGE + '.png',bbox_inches='tight')
        plt.close()
        plt.cla()
        plt.clf()
    except:
        print('Error ' + GAGE)
        pass

<Figure size 842.4x595.44 with 0 Axes>